In [22]:
import json
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import matplotlib.pyplot as plt
from interaction_inference import simulation
from interaction_inference.constraints import Constraint
from interaction_inference.optimization import Optimization
from copy import deepcopy

In [2]:
rng = np.random.default_rng(1)

In [106]:
# simulate sample
data = simulation.simulate_dataset_range_BD(
    name = "test",
    interaction_values = [50, 20, 10, 8, 6, 4, 2, 1, 0.5, 0], #[5, 4.5, 4, 3.5, 3, 2.5, 2, 1.5, 1, 0.5, 0],
    cells = 1000,
    rate = 1,
    tqdm_disable = False
)

100%|██████████| 10/10 [00:07<00:00,  1.28it/s]


In [107]:
# downsample
m = 0.75
b = (1 / m) - 1
if b == 0 :
    beta = np.ones(1000)
else:
    beta = rng.beta(1, b, size=1000)
data_downsampled = data.downsample("test", beta)

In [108]:
# bootstrap
data_downsampled.bootstrap_probabilities(tqdm_disable=False)
data_downsampled.compute_fm(tqdm_disable=False)

100%|██████████| 10/10 [00:00<00:00, 26.55it/s]


In [109]:
# copy data
data_downsampled_point = deepcopy(data_downsampled)
eps = 10 ** -6
for sample, fms in data_downsampled_point.fm_OB.items():
    # replace fm1m2 bounds by mean bound
    fm1m2 = fms['fm1m2']
    for m1 in range(fm1m2.shape[1]):
        for m2 in range(fm1m2.shape[2]):
            if not (fm1m2[0, m1, m2] == 0.0 and fm1m2[1, m1, m2] == 1.0):
                mean = (fm1m2[0, m1, m2] + fm1m2[1, m1, m2]) / 2
                fm1m2[0, m1, m2] = mean - eps
                fm1m2[1, m1, m2] = mean + eps
    data_downsampled_point.fm_OB[sample]['fm1m2'] = fm1m2
    # replace fm1 bounds by mean bound
    fm1 = fms['fm1']
    for m1 in range(fm1.shape[1]):
        if not (fm1[0, m1] == 0.0 and fm1[1, m1] == 1.0):
            mean = (fm1[0, m1] + fm1[1, m1]) / 2
            fm1[0, m1] = mean - eps
            fm1[1, m1] = mean + eps
    data_downsampled_point.fm_OB[sample]['fm1'] = fm1
    # replace fm2 bounds by mean bound
    fm2 = fms['fm2']
    for m2 in range(fm2.shape[1]):
        if not (fm2[0, m2] == 0.0 and fm2[1, m2] == 1.0):
            mean = (fm2[0, m2] + fm1[1, m2]) / 2
            fm2[0, m2] = mean - eps
            fm2[1, m2] = mean + eps
    data_downsampled_point.fm_OB[sample]['fm2'] = fm2

In [110]:
# construct constraints
constraints = Constraint(
    downsampled_probability = True,
    downsampled_CME         = True,
    k_deg_1                 = True,
    k_reg                   = True
)

# optimize
opt = Optimization(data_downsampled, constraints, tqdm_disable=True, print_solution=False)

In [111]:
# construct constraints
constraints = Constraint(
    downsampled_probability = True,
    downsampled_CME         = True,
    k_deg_1                 = True,
    k_reg                   = True
)

# optimize
opt_point = Optimization(data_downsampled_point, constraints, tqdm_disable=True, print_solution=False)

In [112]:
for i in opt.result_dict.keys():
    print(f"{opt.result_dict[i]['status']}, {opt_point.result_dict[i]['status']}")

OPTIMAL, OPTIMAL
INFEASIBLE, INFEASIBLE
INFEASIBLE, INFEASIBLE
OPTIMAL, OPTIMAL
INFEASIBLE, INFEASIBLE
INFEASIBLE, INFEASIBLE
OPTIMAL, OPTIMAL
OPTIMAL, OPTIMAL
OPTIMAL, OPTIMAL
OPTIMAL, OPTIMAL
